kodların düzenlenmesi gerekiyor


In [ ]:
# !pip install plotly==4.7.1 #Installing plotly
# !wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -q -O /usr/local/bin/orca
# !chmod +x /usr/local/bin/orca
# !apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [1]:
import json #To read and process the Json files
import os #To file operations
import numpy as np #For data manipulation and analysis
import pandas as pd #For data manipulation and analysis
import cv2 #To process images
import plotly.express as px #To visualize the data
import plotly.graph_objects as go #To visualize the data
from tqdm import tqdm #To see the progress
from natsort import natsorted #To sort the data considering the index
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Two expert labeled the sample frames and upload to Github. Labeled frames are in JSON format.
adress = 'https://raw.githubusercontent.com/kynemre/CellSegmentation/master/DATA/jsonfiles.zip';#JSON files path
!wget -q -O data.zip "{adress}";# Download data as ZIP
!unzip -q data.zip # Extract ZIP file
!rm data.zip ## Delete ZIP file

replace frame1.jpg.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
# This code block is to upload different data from your computer (labeled frames) in JSON format.
#uploaded = files.upload() #upload your own JSON

In [3]:
df = pd.DataFrame(columns=['Name','Frame',
                           'Area','Ccf','Circularity',
                           'LocX','LocY', 'Velocity', 
                           'Total Distance Traveled']) #Create an empty dataframe


paths = list(filter(lambda x:x.endswith(".json"),os.listdir("/content"))) #Read files and get only JSON files
cell_names = list() 
frame_numbers = list()
for path in tqdm(paths):#Loop for each JSON
    with open(path) as f: #Open the JSON file
        data = json.load(f) #Read the JSON 

        frame_value = data["tags"][0]["value"] #Get frame number
        if frame_value not in frame_numbers: # Add frame to frame list
            frame_numbers.append(frame_value)
        HEIGHT = data["size"]["height"] #Get the height of output from data
        WIDTH = data["size"]["width"] #Get the width of output from data


        for i in data["objects"]: #Draw each object(cell)
            img = np.zeros((HEIGHT,WIDTH)) #Create a canvas

            tag = i["tags"][0]["name"] #Get cell tag
            
            contours = np.array(i["points"]["exterior"])#Get the corners
            
            cv2.fillPoly(img, pts =[contours], color=(255,255,255))#Add the object to the canvas
            img[img!=0] = 1 #Convert img to binary (0 and 1)
            area = np.sum(img)*3 #Calculate area of the cell
##############################################################################################################################
            x_total = np.where(img==1)[1] #Sum all x coordinates
            #Finding average x coordinate
            x = np.sum(x_total)/len(x_total) 
            y_total = np.where(img==1)[0] #Sum all y coordinates
            #Finding average y coordinate
            y = HEIGHT - np.sum(y_total)/len(y_total) 

            base = np.zeros((HEIGHT+50,WIDTH+50)) #Create an empty canvas
            #Place the image in the center of the canvas
            base[25:HEIGHT+25,25:WIDTH+25] = img 
            #Finding the edges of the cell
            base = abs(np.gradient(base)[0]) + abs(np.gradient(base)[1]) 
            base[base!=0] = 1 #Make corrections
            perimeter = np.sum(base) #Find the perimeter length of the cell
            
            #Finding the circularity of the cell
            circularity = 4*np.pi*area/(perimeter**2) 

            if tag not in cell_names: #Save the names of cells
                cell_names.append(tag)
            
            _df = pd.DataFrame({'Name':tag,
                         'Frame':frame_value,
                         'Area':area,
                         'Ccf':perimeter,
                         'Circularity': circularity,
                         'LocX':x,
                         'LocY':y
                         }, index=[0])
            df = df.append(_df, ignore_index = True)#Add data to our table
cell_names = natsorted(cell_names) #Sorting
frame_numbers.sort()

100%|██████████| 15/15 [00:33<00:00,  2.22s/it]


In [4]:
#Fix in dataset for cells that are off-screen in some frames
for cell_name in cell_names:
    for frame_value in frame_numbers:
        if frame_value not in list(df[df.Name == cell_name]['Frame']):
            df = df.append(pd.DataFrame({
                'Name':cell_name,
                'Frame':frame_value,
                'Area':1,
                'Ccf':0,
                'Circularity': 0.001
            }, index=[0]), ignore_index = True)

In [5]:
#In this code block we find the displacement of cells

dff = df.fillna(value=0)
for cell_name in cell_names:
    previous_way_taken = 0
    for i in range(1,len(frame_numbers)):

        x_ilk = float(dff[(dff.Name == cell_name) & 
                          (dff.Frame == frame_numbers[i-1])]['LocX'])
        y_ilk = float(dff[(dff.Name == cell_name) & 
                          (dff.Frame == frame_numbers[i-1])]['LocY'])

        x_son = float(dff[(dff.Name == cell_name) & 
                          (dff.Frame == frame_numbers[i])]['LocX'])
        y_son = float(dff[(dff.Name == cell_name) & 
                          (dff.Frame == frame_numbers[i])]['LocY'])

        if df[(df.Name == cell_name) & 
              (df.Frame == frame_numbers[i-1])]['LocX'].isnull().any().any():
            velocity = np.nan
            if df[(df.Name == cell_name) & 
                  (df.Frame == frame_numbers[i])]['LocX'].isnull().any().any():
                total_taken_way = previous_way_taken
        else:
            velocity = (((x_son-x_ilk)**2+(y_son-y_ilk)**2)**0.5)
            if df[(df.Name == cell_name) & 
                  (df.Frame == frame_numbers[i-1])]['Total Distance Traveled'].isnull().any().any():
                total_taken_way = velocity
            if df[(df.Name == cell_name) & 
                  (df.Frame == frame_numbers[i])]['LocX'].isnull().any().any():
                total_taken_way = previous_way_taken
            if (df[(df.Name == cell_name) & 
                  (df.Frame == frame_numbers[i-1])]['Total Distance Traveled'].isnull().any().any() == False) and (df[(df.Name == cell_name) & 
                      (df.Frame == frame_numbers[i])]['LocX'].isnull().any().any()==False):
                total_taken_way = velocity + float(df[(dff.Name == cell_name) & 
                                                      (df.Frame == frame_numbers[i-1])]['Total Distance Traveled'])
                previous_way_taken = total_taken_way

        df.loc[(df.Name == cell_name) & 
               (df.Frame == frame_numbers[i]),'Total Distance Traveled'] = total_taken_way
        df.loc[(df.Name == cell_name) & 
               (df.Frame == frame_numbers[i]),'Velocity'] = velocity
        
df.loc[df['LocX'].isnull(),'Velocity' ] = np.nan
df.loc[df.Velocity == 0,'Velocity'] = np.nan

In [ ]:
#Sorting
df['sort'] = df['Name'].str.extract('(\d+)', expand=False).astype(int)
df.sort_values(['Frame','sort'],inplace=True, ascending=True)
df = df.drop('sort', axis=1)

In [ ]:
fig = px.scatter(df.fillna(value=1000),
                 x="LocX", 
                 y="LocY", 
                 animation_frame="Frame",
                 size="Area", 
                 color="Name", 
                 hover_name="Name",
                 size_max=45,
                 title='Cell Movement',
                 range_x=[0,max(df['LocX'])],
                 range_y=[0,max(df['LocY'])],
                 width = WIDTH/2.5,
                 height = HEIGHT/2.5)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 750
fig.show();

In [ ]:
fig = go.Figure(layout=go.Layout(
    xaxis=dict(title='x-axis (px)'),
    yaxis=dict(title='y-axis (px)'),
    title='Cell Positions',
    width = WIDTH/2.6,
    height = HEIGHT/2.6))

colors = ['#f95','#cc0','#f0f','#0c0','#0cc','#00f',
          '#f00','#d00','#a00','#d51','#337']

for i,name in enumerate(cell_names):
    if not name in ['Cell_10', 'Cell_11']:
        fig.add_trace(go.Scatter(x=df[df["Name"]==name]["LocX"],
                                y=df[df["Name"]==name]["LocY"],
                                mode='lines',
                                text=df[df["Name"]==name]["Frame"],
                                line=dict(color=colors[i]),
                                name=name))    
fig.show()

In [ ]:
import plotly
fig = go.Figure(layout=go.Layout(
    xaxis=dict(title='Frame'),
    yaxis=dict(title='Area (px)'),
    title='Areas of cells'))



for name in cell_names:
    if not name in ['Cell_10', 'Cell_11', 'Cell_7A', 'Cell_7B']: 
        fig.add_trace(go.Scatter(x=df[df["Name"]==name]["Frame"],
                                y=df[df["Name"]==name]["Area"],
                                mode='markers+lines',
                                name=name))
fig.show()

In [ ]:
fig = go.Figure(layout=go.Layout(
    xaxis=dict(title='Frame'),
    yaxis=dict(title='Circularity'),
    title='Circularity of Cells'))

for name in cell_names:
    if not name in ['Cell_10', 'Cell_11', 'Cell_7A', 'Cell_7B']:
        fig.add_trace(go.Scatter(x=df[df["Name"]==name]["Frame"],
                                y=df[df["Name"]==name]["Circularity"],
                                mode='markers+lines',
                                name=name))
fig.show()

In [ ]:
fig = go.Figure(layout=go.Layout(
    xaxis=dict(title='Frame'),
    yaxis=dict(title='Velocity (px)'),
    title="Displacement of Cells relative to the Previous Frame"))

for name in cell_names:
    if not name in ['Cell_10', 'Cell_11', 'Cell_7A', 'Cell_7B']:
        fig.add_trace(go.Scatter(x=df.dropna()[df.dropna()["Name"]==name]["Frame"],
                                y=df.dropna()[df.dropna()["Name"]==name]["Velocity"],
                                mode='markers+lines',
                                name=name))
fig.show()

In [ ]:
fig = px.bar(df, 
             x="Name", 
             y="Total Distance Traveled", 
             color="Name",
             animation_frame="Frame",
             range_y=[0,1150],
             title='Total Distance Traveled for Each Cell')
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 750
fig.update_layout(showlegend=False)
fig.show();

In [ ]:
max_v = df[['Name', 'Frame', 'LocX','LocY']]
max_v = max_v.dropna()
max_v.sort_values(by='Frame')
data_v = dict()
for cell_name in natsorted(cell_names):
    x_last = max_v[max_v['Name']==cell_name].iloc[-1]['LocX']
    x_first = max_v[max_v['Name']==cell_name].iloc[0]['LocX']

    y_last = max_v[max_v['Name']==cell_name].iloc[-1]['LocY']
    y_first = max_v[max_v['Name']==cell_name].iloc[0]['LocY']

    data_v[cell_name] = ((x_last-x_first)**2+(y_last-y_first)**2)**0.5

fig = px.bar(pd.DataFrame(data=data_v.items(), columns=['Name', 'Replacement']),
             x='Name', 
             y='Replacement',
             labels={'Name':'Cell Names', 'Replacement':'Displacement'},
             title='Total Displacement for Each Cell')
fig.update_layout(xaxis_tickangle=-45)
fig.show()